In [95]:
# Api used jupyter notebook
# Genre - genre_id, genre_name   (formally subcategories)
# Event_Capacity - event_id, quantity_sold, quantity_pending, quantity_total
# Event- event id, summary, category_id, event_name, start_date, end_date, URL, status?, online, age_restriction?, 
# Ticket_Classes - event_id, description, capacity, quantity_sold
# Venue - event_id, venue_id, only_public, status, venue_name, age_restriction, city, region, postal_code, (latitude, longitude for the map)

In [96]:
#imports
import pandas as pd
import requests 

path = ('Project03_Visualization_SHOWWW\csv_tables')

In [105]:
#base url
#url =  'https://app.ticketmaster.com/discovery/v2/events.json?countryCode=US&classificationName=Music&startDateTime=2023'
genre_url = 'https://app.ticketmaster.com/discovery/v2/classifications/KZFzniwnSyZfZ7v7nJ.json?apikey=yTIm5zlYHTsXuGFluFTMc9u5peB89aGA'

In [ ]:
#getting genre from ticketmaster developer
# using separate api pull for simple list of music genres

genre_call = requests.get(genre_url)
genre_call_json = genre_call.json()
# print(genre_json)
subgenre_json = genre_call_json['segment']['_embedded']['genres'][4]['_embedded']['subgenres']
genre_json = genre_call_json['segment']['_embedded']['genres']
# print(subgenre_json)
# print(genre_json)

genre_list = []
subgenre_list = []

for genre in genre_json:
    genre_id = genre['id']
    genre_name = genre['name']
    subgenres = genre['_embedded']['subgenres']

    for subgenre in subgenres:
        subgenre_name = subgenre['name']
        subgenre_id = subgenre['id']
        genre_list.append({'genre': genre_name, 'id': genre_id, 'subgenre': subgenre_name, 'subgenre_id': subgenre_id})
        subgenre_list.append(subgenre_name)

subgenre_list
genre_list


In [104]:
genre_df = pd.json_normalize(genre_list)
genre_df.to_csv('csv_tables/genre.csv')

In [109]:
base_url =  'https://app.ticketmaster.com/discovery/v2/events.json?countryCode=US&classificationName=Music&startDateTime=2023-06-20T00:00:00Z&sort=date,asc&apikey=rmkU0um9gY6pMG0Ti7M0AFnq4KTtVwGd'

event_call = requests.get(base_url)
event_call_json = event_call.json()

event_json = event_call_json['_embedded']['events']

event_data = []
genre_id = []
subgenre_id = []
seatMapUrl_list = []
venueURL_list = []
venue_data = []
for event in event_json:
    event_name = event['name']
    event_id = event['id']
    event_url = event['url']
    image_url = event['images'][0]['url']
    startDate = event['dates']['start']['localDate']
    startTime = event['dates']['start']['localTime']
    classifications = event['classifications']
    venues = event["_embedded"]['venues']
    event_data.append({"Event Name" : event_name,
                       "Event ID" : event_id,
                       "Event URL" : event_url,
                       "Image URL" : image_url,
                       "Start Date" : startDate,
                       "Start Time" : startTime
                       })
    for venue in venues:
        venue_name = venue['name']
        venue_id = venue['id']
        venue_postal = venue['postalCode']
        venue_city = venue['city']
        venue_state = venue['state']['stateCode']
        venue_add = venue['address']
        venue_geo = venue['location']
        venue_url = venue.get('url', '')  # Use .get() with a default value if 'url' key is missing
        venueURL_list.append(venue_url)
        venue_data.append({"Venue Name": venue_name, 
                           "Venue ID": venue_id, 
                           "Postal Code" : venue_postal, 
                           "City" : venue_city, 
                           "State" : venue_state,
                           "Address" : venue_add,
                           "Latitude and Longitude" : venue_geo
                           })
    seatMap = event.get('seatmap')
    if seatMap:
        seatMapUrl_list.append(seatMap['staticUrl'])
    
    for classification in classifications:
        genre = classification['genre']
        subgenre = classification.get('subGenre', '')  # Use .get() to safely retrieve the value
        if subgenre:
            subgenre_id.append(subgenre['id'])
        genre_id.append(genre['id'])
